In [1]:
import csv
import os
import re
import numpy as np
import time

In [2]:
from dotenv import load_dotenv
load_dotenv()

#grab hidden values
USERNAME_PRO = os.environ.get('USERNAME')
PASSWORD_USER = os.environ.get('PASSWORD')
CLIENT_REDD = os.environ.get('CLIENT')
CLIENT_PASS = os.environ.get('CLIENTPASS')

In [3]:
# Step 3.3: import this module and log in!
import praw
reddit = praw.Reddit(client_id= CLIENT_REDD, # App ID
                     client_secret= CLIENT_PASS, # App password
                     user_agent='My Test Reddit Scraper', # App name (can be different)
                     username= USERNAME_PRO, # Username
                     password= PASSWORD_USER) # User password

In [4]:
# To get a specific subreddit
subreddit = reddit.subreddit('wallstreetbets')
print(subreddit.display_name)  # Displayed name
print(subreddit.title)         # "Full" name
print(subreddit.description)   # Description

wallstreetbets
wallstreetbets
Our official Twitter: [@Official_WSB](https://twitter.com/official_wsb)

---

The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/wallstreetbets)

[Appeal a Discord ban](https://discord.gg/S9vH92cQ6W)


**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbe

In [7]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kaimoy/nltk_data...


True

In [ ]:
import vaderSentiment as vader

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [11]:
#run through dataset and store comment data inot lists


from datetime import datetime
file_list = []
comment_id_list = []
comment_date_list=[]
comment_body_list=[]

#UNIX time for the minimum post date-December 15th, 2021
submission_lower_date = 1608033086
#UNIX time for the maximum post date-February 15th, 2021 
submission_cap_date = 1613393219
#search for keyword gamestop or gme 
for thread in subreddit.search(query="gamestop gme".lower()):
    if(thread.created_utc > submission_lower_date and thread.created_utc < submission_cap_date):
            t = thread.created_utc
            dt = datetime.fromtimestamp(t)
            
            file_list.append(thread)
            comments = thread.comments.list()
            for comment in comments:
                try:
                    comment_id_list.append(comment.id)
                    comment_date_list.append(comment.created_utc)
                    comment_body_list.append(comment.body)
                    analyzer(comment)
                   # print(comment.id)
                    #print(comment.created_utc)
                    #print(comment.body)
                    #print()
                except:
                    pass
print(len(comment_id_list))
print(len(file_list))



7218
43


In [12]:
#run vader sentiment analysis for each comment body
#store ONLY vader compound score into list vadercompound_list
vs_list=[]
sentiment_list=[]
analyzer = SentimentIntensityAnalyzer()
vadercompound_list = []
for comment in comment_body_list:
        vs = analyzer.polarity_scores(comment)
        compound_score = vs["compound"]
        sentiment = ""
        if(compound_score > 0.05):
            sentiment = "Positive"
        elif(compound_score < -0.05):
            sentiment = "Negative"
        else:
            sentiment = "Neutral"
        vs_list.append(vs)
        sentiment_list.append(sentiment)
        vadercompound_list.append(compound_score)
    

In [21]:
#prints comment date, body, score
print("Comment Body:",comment_body_list[40], "\n","Time:", comment_date_list[40],"\nScore:",vadercompound_list[40])

Comment Body: They’ll be losing a lot of users 
 Time: 1611852349.0 
Score: -0.3818


In [22]:
print("Comment Body:",comment_body_list[40],"\n")
print("Vader Scores:",str(vs_list[40]),"\n")
print("Sentiment:",sentiment_list[40])

Comment Body: They’ll be losing a lot of users 

Vader Scores: {'neg': 0.302, 'neu': 0.698, 'pos': 0.0, 'compound': -0.3818} 

Sentiment: Negative


In [23]:
#1 day = 86400 seconds/unix
day = 86400
start_time=1608033086    #first observation day, specified above 
end_time =0
final_time =1613393219 #last day of market observation
duration = 63

#list for positive vader scored comments total
pos_comment=[]
#list for positive vader scored comments total
neg_comment=[]
#list for positive vader scored comments total
neut_comment=[]
#total comments in duration
total_comment=[]
daily_avg_pos=[]
daily_avg_neg=[]
daily_avg_neut=[]

#end time is always one day ahead of start time
end_time = start_time + day
for t in range(duration):
    #list of total comments per day
    daily_total=[]
    #list of comment times that resets for each day 
    daily_pos=[]
    daily_neg=[]
    daily_neut=[]
    scored_pos=0
    scored_neg=0
    scored_neut=0
    for i in range(len(comment_date_list)):
        if(comment_date_list[i] > start_time and comment_date_list[i] < end_time):
            if(vadercompound_list[i] > 0.05):
                daily_pos.append(comment_date_list[i])
                scored_pos+=vadercompound_list[i]
                #print(vadercompound_list[i])
            elif(vadercompound_list[i] < -0.05):
                daily_neg.append(comment_date_list[i])
                scored_neg+=vadercompound_list[i]
            else:
                daily_neut.append(comment_date_list[i])
                scored_neut+=vadercompound_list[i]
                
            daily_total.append(comment_date_list[i])
    if((len(daily_pos))>0):
        daily_avg_pos.append(scored_pos/(len(daily_pos)))
    else:
        daily_avg_pos.append(0)
    if((len(daily_neg))>0):
        daily_avg_neg.append(scored_neg/(len(daily_neg)))
    else:
        daily_avg_neg.append(0)
    if((len(daily_neut))>0):
        daily_avg_neut.append(scored_neut/(len(daily_neut)))
    else:
        daily_avg_neut.append(0)
    
    end_time += day
    start_time +=day
    pos_comment.append(len(daily_pos))
    neg_comment.append(len(daily_neg))
    neut_comment.append(len(daily_neut))
    total_comment.append(len(daily_total))



In [24]:
print(daily_avg_pos)

[0, 0, 0, 0, 0, 0.524394366197183, 0.5305285714285715, 0.4215, 0.5361361538461534, 0.5469872727272728, 0.5725258064516129, 0, 0, 0, 0.6048166666666667, 0.45149999999999996, 0, 0, 0.5952355555555556, 0.5803908163265304, 0.5418858823529411, 0.4833, 0.513257894736842, 0.29569999999999996, 0.8481, 0, 0, 0.5559725663716816, 0.5196026315789473, 0.5755791666666668, 0.46640714285714285, 0.44540588235294126, 0.5684333333333333, 0.689675, 0.5657732142857136, 0.54323, 0.5222853658536586, 0.5530391304347827, 0.58933, 0.35342, 0, 0.466, 0.5094591743119258, 0.49846153846153857, 0.583409090909091, 0.5386295566502458, 0.564871186440678, 0.5255833333333333, 0.5434177777777779, 0.32729230769230766, 0.5849769230769231, 0.35435, 0.638, 0.3355, 0.5282015789473685, 0.461525, 0.63915, 0.47529047619047615, 0.46727500000000005, 0.578252, 0.47961, 0, 0]


In [25]:
#lists the total frequency of comments per day
print(total_comment)

[0, 0, 0, 0, 0, 140, 94, 5, 308, 124, 41, 1, 1, 0, 26, 13, 0, 0, 72, 244, 157, 55, 34, 6, 2, 0, 0, 190, 185, 51, 546, 68, 24, 6, 417, 39, 98, 88, 86, 7, 3, 14, 533, 172, 29, 470, 704, 587, 101, 44, 81, 12, 18, 7, 390, 6, 2, 58, 14, 117, 16, 2, 0]


In [26]:
import csv

In [28]:
import pandas as pd

In [29]:
ground_truth=[]

with open(r"gmeChange - Sheet1.csv",newline ='') as csvfile:
    csvReader = csv.reader(csvfile)
    for row in csvReader:
        ground_truth.append(row)

print(ground_truth)

[['0.0888'], ['0.01'], ['0.0708'], ['0.0539'], ['0'], ['0'], ['-0.0064'], ['0.2531'], ['0.057'], ['-0.0204'], ['0'], ['0'], ['0'], ['0.0417'], ['-0.0767'], ['-0.0062'], ['-0.0218'], ['0'], ['0'], ['0'], ['-0.0844'], ['0.007'], ['0.057'], ['-0.0153'], ['-0.0216'], ['0'], ['0'], ['0.1272'], ['0.0005'], ['0.5739'], ['0.271'], ['-0.1105'], ['0'], ['0'], ['0'], ['0.1087'], ['-0.0061'], ['0.099'], ['0.5108'], ['0'], ['0'], ['0.1812'], ['0.9272'], ['1.3484'], ['-0.4429'], ['0.6787'], ['0'], ['0'], ['-0.3077'], ['-0.6'], ['0.0268'], ['-0.4211'], ['0.192'], ['0'], ['0'], ['-0.0591'], ['-0.1615'], ['0.0177'], ['-0.002'], ['0.0254'], ['0'], ['0'], ['0']]


In [31]:
print(len(ground_truth))

63


In [37]:
df_truth = pd.DataFrame(ground_truth)

In [65]:
df_arr = np.asarray(ground_truth)

In [68]:
oneD_truth = df_arr.reshape(-1)

In [85]:
float_truth =np.array(oneD_truth,dtype=float)

In [38]:
df_truth

,0
0,0.0888
1,0.01
2,0.0708
3,0.0539
4,0
...,...
58,-0.002
59,0.0254
60,0
61,0


In [32]:
#create linear regression model 
from sklearn.linear_model import LinearRegression
import numpy as np

In [86]:
#double brackets turns it from array into matrix
X = []
#x0=[[]]
#x1=[[]]
#x2=[[]]

#x0 = np.asarray(x0) # do for all

In [87]:
constant = []
for i in range(len(oneD_truth)):
    constant.append(0.08205121951)
print(len(constant))

63


In [77]:
len(constant)

63

In [88]:
Y = float_truth
Y = np.asarray(Y)
#Constant Variable
x0 = [constant]
#Count of Positive Comments on a given day 
x1 = [pos_comment]
#Count of Negative Comments
x2 = [neg_comment]
x3 = [daily_avg_pos]
x4 = [daily_avg_neg]
x0 = np.asarray(x0)
x1 = np.asarray(x1)
x2 = np.asarray(x2)
x3 = np.asarray(x3)
x4 = np.asarray(x4)
print(Y)
print(x0)
print(x1)
print(x2)
print(x3)
print(x4)

[ 8.8800e-02  1.0000e-02  7.0800e-02  5.3900e-02  0.0000e+00  0.0000e+00
 -6.4000e-03  2.5310e-01  5.7000e-02 -2.0400e-02  0.0000e+00  0.0000e+00
  0.0000e+00  4.1700e-02 -7.6700e-02 -6.2000e-03 -2.1800e-02  0.0000e+00
  0.0000e+00  0.0000e+00 -8.4400e-02  7.0000e-03  5.7000e-02 -1.5300e-02
 -2.1600e-02  0.0000e+00  0.0000e+00  1.2720e-01  5.0000e-04  5.7390e-01
  2.7100e-01 -1.1050e-01  0.0000e+00  0.0000e+00  0.0000e+00  1.0870e-01
 -6.1000e-03  9.9000e-02  5.1080e-01  0.0000e+00  0.0000e+00  1.8120e-01
  9.2720e-01  1.3484e+00 -4.4290e-01  6.7870e-01  0.0000e+00  0.0000e+00
 -3.0770e-01 -6.0000e-01  2.6800e-02 -4.2110e-01  1.9200e-01  0.0000e+00
  0.0000e+00 -5.9100e-02 -1.6150e-01  1.7700e-02 -2.0000e-03  2.5400e-02
  0.0000e+00  0.0000e+00  0.0000e+00]
[[0.08205122 0.08205122 0.08205122 0.08205122 0.08205122 0.08205122
  0.08205122 0.08205122 0.08205122 0.08205122 0.08205122 0.08205122
  0.08205122 0.08205122 0.08205122 0.08205122 0.08205122 0.08205122
  0.08205122 0.08205122 0.08

In [89]:
X = np.concatenate((x0.T,x1.T,x2.T,x3.T,x4.T),axis=1)
print(X)

[[ 8.20512195e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.20512195e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.20512195e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.20512195e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.20512195e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.20512195e-02  7.10000000e+01  1.70000000e+01  5.24394366e-01
  -4.65829412e-01]
 [ 8.20512195e-02  3.50000000e+01  1.80000000e+01  5.30528571e-01
  -4.49883333e-01]
 [ 8.20512195e-02  1.00000000e+00  1.00000000e+00  4.21500000e-01
  -5.42300000e-01]
 [ 8.20512195e-02  1.30000000e+02  7.10000000e+01  5.36136154e-01
  -5.01336620e-01]
 [ 8.20512195e-02  5.50000000e+01  3.10000000e+01  5.46987273e-01
  -4.82138710e-01]
 [ 8.20512195e-02  3.10000000e+01  6.00000000e+00  5.72525806e-01
  -3.58416667e-01]
 [ 8.20512195e-02  0.00000000e+00  1.00000000e+00  0.00000000e+00

In [90]:
reg = LinearRegression().fit(X,Y)
print(reg.score(X,Y))

0.09562173399397855


In [75]:
print(Y)

['0.0888' '0.01' '0.0708' '0.0539' '0' '0' '-0.0064' '0.2531' '0.057'
 '-0.0204' '0' '0' '0' '0.0417' '-0.0767' '-0.0062' '-0.0218' '0' '0' '0'
 '-0.0844' '0.007' '0.057' '-0.0153' '-0.0216' '0' '0' '0.1272' '0.0005'
 '0.5739' '0.271' '-0.1105' '0' '0' '0' '0.1087' '-0.0061' '0.099'
 '0.5108' '0' '0' '0.1812' '0.9272' '1.3484' '-0.4429' '0.6787' '0' '0'
 '-0.3077' '-0.6' '0.0268' '-0.4211' '0.192' '0' '0' '-0.0591' '-0.1615'
 '0.0177' '-0.002' '0.0254' '0' '0' '0']


In [91]:
import statsmodels.api as sm
ols = sm.OLS(Y,X)  
#endog = 1d response variable
ols_result = ols.fit()
print(ols_result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     1.533
Date:                Mon, 03 Oct 2022   Prob (F-statistic):              0.205
Time:                        11:05:31   Log-Likelihood:                -4.4980
No. Observations:                  63   AIC:                             19.00
Df Residuals:                      58   BIC:                             29.71
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0988      0.876     -0.113      0.9